In [1]:
import pandas as pd
from datetime import date
import numpy as np

In [2]:
#same title as old data
#''inspector_badge_number' in pot_insp_sum is mixed type
df_pot_insp_sum = pd.read_csv('raw_data/potential_inspection_summary_data.csv', )
df_pot_vio_det = pd.read_csv('raw_data/potential_violation_details_data.csv')
df_cen_blk = pd.read_csv('census_block_final.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', 40)

In [4]:
pd.options.display.float_format = '{}'.format

In [5]:
df_pot_insp_sum.columns[33]

'inspector_badge_number'

In [6]:
#convert to datetime and drop NaN. Some inspections are missing datetime (NaN). 68363 before drop. 68347 after drop
df_pot_insp_sum['iso_week']= pd.to_datetime(df_pot_insp_sum['inspection_date']).dt.week

In [7]:
df_pot_insp_sum['iso_year'] = pd.to_datetime(df_pot_insp_sum['inspection_date']).dt.year

In [10]:
#16 rows do not have inspection dates. use .dropna(inplace=True)
df_pot_insp_sum['inspection_date'].isnull().sum()

0

In [9]:
df_pot_insp_sum['inspection_date'].dropna(inplace=True)

In [11]:
df_pot_insp_sum['establishment_type'].unique()

array(['Beauty Shop', 'Barber Shop', 'Restaurant Total', 'Bakery',
       'Food Products', 'Delicatessen', 'Beauty Shop-Nails', 'Indoor Pool',
       'Caterer', 'Grocery Store', 'Marine-Food (Wholesale)', 'Hotel',
       'Unlicensed Food', 'Marine-Food Prod (Retail)',
       'Ice Cream Manufacturer', 'School Cafeteria', 'Unknown',
       'Commission Merchant', 'Massage & Spa Treatment',
       'Beauty Shop-Braiding', 'Beauty Shop-Esthetics', 'Health Spa Sales',
       'Mobile Vending', 'Health Spa'], dtype=object)

In [12]:
df_pot_insp_sum['risk_category'].unique()

array([  2.,  nan,   3.,   1.,   4.,   5.])

In [13]:
merge1 = pd.merge(df_pot_insp_sum, df_cen_blk, on='inspection_id')

In [14]:
merge1.head()

,inspection_id,establishment_name,address,telephone,email,inspection_date,inspection_time_in,inspection_time_out,license_holder,license_number,license_period_start,license_period_end,establishment_type,risk_category,inspection_type,total_violations,priority_violations,priority_violations_corrected_on_site,priority_violations_repeated,priority_foundation_violations,priority_foundation_violations_corrected_on_site,priority_foundation_violations_repeated,core_violations,core_violations_corrected_on_site,core_violations_repeated,critical_violations,critical_violations_corrected_on_site,critical_violations_repeated,noncritical_violations,noncritical_violations_corrected_on_site,noncritical_violations_repeated,inspector_comments,inspector_name,inspector_badge_number,known_valid,iso_week,iso_year,GEOID10,latitude,longitude
0,1,Tigani Hair Boutique,"2130 P ST NW WASHINGTON, DC 20037",(202) 466-2626,NaN,2009-01-29,NaN,NaN,Tigani P Street LLC,09202xxxx-67003480,2013-06-01,2015-05-31,Beauty Shop,2.0,Routine,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False,5.0,2009.0,110010055004000.0,38.909636,-77.047744
1,2,Chevy Chase Arcade Barber Shop,"5520 CONNECTICUT AVE NW WASHINGTON, DC 20015",NaN,NaN,2008-09-26,NaN,NaN,Chevy Chase Arcade Barber Shop,NaN,NaN,NaN,Barber Shop,nan,Routine,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NF,NaN,False,39.0,2008.0,110010011001009.0,38.963665,-77.0749
2,3,Chevy Chase Arcade Barber Shop,"5520 CONNECTICUT AVE NW WASHINGTON, DC 20015",NaN,NaN,2008-11-20,NaN,NaN,Chevy Chase Arcade Barber Shop,NaN,NaN,NaN,Barber Shop,nan,Follow-up,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NF,NaN,False,47.0,2008.0,110010011001009.0,38.963665,-77.0749
3,4,Chevy Chase House,"1845 M ST NE WASHINGTON, DC 20002",(202) 686-0045,NaN,2009-02-18,NaN,NaN,Chevy Chase House,NaN,2011-05-20,2011-06-20,Restaurant Total,3.0,Other,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AG,NaN,False,8.0,2009.0,110010089031005.0,38.905549,-76.976949
4,5,SALINA RESTAURANT,"1936 9TH ST NW WASHINGTON, DC 20001",(202) 299-0511,terfneh.kahsay@gmail.com,2009-09-18,NaN,NaN,Salina Corporation,09313xxxx-12000109,2009-09-01,2011-08-31,Restaurant Total,2.0,Preoperational,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,NaN,CB,NaN,False,38.0,2009.0,110010044002015.0,38.914838,-77.02404200000002


In [15]:
df_pot_vio_det['rat_violation'] = (df_pot_vio_det['violation_description'] == 'Insects, rodents, & animals not present') | (df_pot_vio_det['violation_description'] == 'Insects, rodents, and animals not present')

In [16]:
#number of violation descriptions that are rat related
df_pot_vio_det['rat_violation'].sum()

8988

In [17]:
#3536 w/and. 5452 w/&
len(df_pot_vio_det[df_pot_vio_det['violation_description']=='Insects, rodents, & animals not present'])

5452

In [18]:
merge2 = pd.merge(merge1, df_pot_vio_det, on='inspection_id')

In [19]:
merge2.columns

Index(['inspection_id', 'establishment_name', 'address', 'telephone', 'email',
       'inspection_date', 'inspection_time_in', 'inspection_time_out',
       'license_holder', 'license_number', 'license_period_start',
       'license_period_end', 'establishment_type', 'risk_category',
       'inspection_type', 'total_violations', 'priority_violations',
       'priority_violations_corrected_on_site', 'priority_violations_repeated',
       'priority_foundation_violations',
       'priority_foundation_violations_corrected_on_site',
       'priority_foundation_violations_repeated', 'core_violations',
       'core_violations_corrected_on_site', 'core_violations_repeated',
       'critical_violations', 'critical_violations_corrected_on_site',
       'critical_violations_repeated', 'noncritical_violations',
       'noncritical_violations_corrected_on_site',
       'noncritical_violations_repeated', 'inspector_comments',
       'inspector_name', 'inspector_badge_number', 'known_valid', 'iso_wee

In [20]:
merge2.head()

,inspection_id,establishment_name,address,telephone,email,inspection_date,inspection_time_in,inspection_time_out,license_holder,license_number,license_period_start,license_period_end,establishment_type,risk_category,inspection_type,total_violations,priority_violations,priority_violations_corrected_on_site,priority_violations_repeated,priority_foundation_violations,...,critical_violations,critical_violations_corrected_on_site,critical_violations_repeated,noncritical_violations,noncritical_violations_corrected_on_site,noncritical_violations_repeated,inspector_comments,inspector_name,inspector_badge_number,known_valid,iso_week,iso_year,GEOID10,latitude,longitude,violation_number,violation_description,violation_text,dcmr_25_code,rat_violation
0,292,Pat's Stand #12,"1263 4TH ST NE Washington, DC 20002",(202) 548-0070,NaN,2010-12-14,12:15 PM,01:00 PM,Patricia A. Robinson,25096xxxx-68000864,2010-02-01,2012-01-31,Grocery Store,2.0,Routine,2,nan,nan,nan,nan,...,0.0,0.0,0.0,2.0,0.0,0.0,CFS PATRICIA A ROBINSON 37477 EXP 10/29/2011 F...,CHARLES BASSEY,612,False,50.0,2010.0,110010088031024.0,38.908138,-76.999713,45,"Food and nonfood-contact surfaces cleanable, p...","Dirty cabinets, in storage area.",1700.1,False
1,292,Pat's Stand #12,"1263 4TH ST NE Washington, DC 20002",(202) 548-0070,NaN,2010-12-14,12:15 PM,01:00 PM,Patricia A. Robinson,25096xxxx-68000864,2010-02-01,2012-01-31,Grocery Store,2.0,Routine,2,nan,nan,nan,nan,...,0.0,0.0,0.0,2.0,0.0,0.0,CFS PATRICIA A ROBINSON 37477 EXP 10/29/2011 F...,CHARLES BASSEY,612,False,50.0,2010.0,110010088031024.0,38.908138,-76.999713,53,"Physical facilities: installed, maintained, & ...","DIRTY FLOORS,LITERS IN SERVICE AREA",2900.1,False
2,485,Sugar,"1803 WEST VIRGINIA AVE NE Washington, DC 20002",(202) 832-8887,NaN,2011-03-17,11:50 AM,01:30 PM,"Sugar, LLC",26433xxxx-69006759,2009-08-01,2011-07-31,Delicatessen,3.0,Routine,5,nan,nan,nan,nan,...,2.0,1.0,0.0,3.0,0.0,0.0,FOR QUESTIONS CALL MS COLEMAN AT 202-442-5928 ...,C BASSEY,612,False,11.0,2011.0,110010062021045.0,38.89385,-77.044086,21,Proper date marking & disposition,FOOD IN WALK IN NOT LABELED AS TO DATE OF PREP...,1007.1,False
3,485,Sugar,"1803 WEST VIRGINIA AVE NE Washington, DC 20002",(202) 832-8887,NaN,2011-03-17,11:50 AM,01:30 PM,"Sugar, LLC",26433xxxx-69006759,2009-08-01,2011-07-31,Delicatessen,3.0,Routine,5,nan,nan,nan,nan,...,2.0,1.0,0.0,3.0,0.0,0.0,FOR QUESTIONS CALL MS COLEMAN AT 202-442-5928 ...,C BASSEY,612,False,11.0,2011.0,110010062021045.0,38.89385,-77.044086,34,Thermometers provided & accurate,THERMOMETERS NOT IN ALL COOLING UNITS PROVIDE ...,1510.1,False
4,485,Sugar,"1803 WEST VIRGINIA AVE NE Washington, DC 20002",(202) 832-8887,NaN,2011-03-17,11:50 AM,01:30 PM,"Sugar, LLC",26433xxxx-69006759,2009-08-01,2011-07-31,Delicatessen,3.0,Routine,5,nan,nan,nan,nan,...,2.0,1.0,0.0,3.0,0.0,0.0,FOR QUESTIONS CALL MS COLEMAN AT 202-442-5928 ...,C BASSEY,612,False,11.0,2011.0,110010062021045.0,38.89385,-77.044086,49,Plumbing installed; proper backflow devices,NON FUNCTIONING HAND SINK IN KITCHEN(CORRECT V...,2401.1,False


In [37]:
merge2['rat_violation'].sum()

11293

In [42]:
#clean up final merge
df_clean = merge2[['establishment_type','risk_category', 'GEOID10','iso_week', 'iso_year', 'rat_violation']]

In [43]:
df_clean.head()

,establishment_type,risk_category,GEOID10,iso_week,iso_year,rat_violation
0,Grocery Store,2.0,110010088031024.0,50.0,2010.0,False
1,Grocery Store,2.0,110010088031024.0,50.0,2010.0,False
2,Delicatessen,3.0,110010062021045.0,11.0,2011.0,False
3,Delicatessen,3.0,110010062021045.0,11.0,2011.0,False
4,Delicatessen,3.0,110010062021045.0,11.0,2011.0,False


In [44]:
df_clean.dtypes

establishment_type     object
risk_category         float64
GEOID10               float64
iso_week              float64
iso_year              float64
rat_violation            bool
dtype: object

In [45]:
#GEOID10 has 3,400 null = 1.2% of observations
#risk_category has 143 null
df_clean.isnull().sum()

establishment_type       0
risk_category          143
GEOID10               3395
iso_week                 0
iso_year                 0
rat_violation            0
dtype: int64

In [46]:
df_clean.dropna(inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
len(df_clean), 'GEOID10','iso_week', 'iso_year'

(261441, 'GEOID10', 'iso_week', 'iso_year')

In [48]:
df_clean['risk_category'] = df_clean['risk_category'].astype(int)
df_clean['GEOID10'] = df_clean['GEOID10'].astype(int)
df_clean['iso_week'] = df_clean['iso_week'].astype(int)
df_clean['iso_year'] = df_clean['iso_year'].astype(int)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [49]:
df_clean.head()

,establishment_type,risk_category,GEOID10,iso_week,iso_year,rat_violation
0,Grocery Store,2,110010088031024,50,2010,False
1,Grocery Store,2,110010088031024,50,2010,False
2,Delicatessen,3,110010062021045,11,2011,False
3,Delicatessen,3,110010062021045,11,2011,False
4,Delicatessen,3,110010062021045,11,2011,False


In [50]:
len(df_clean)

261441

In [51]:
df_final = df_clean[df_clean['rat_violation'] == True]

In [52]:
len(df_final)

11246

In [53]:
group = df_final.groupby(['establishment_type','risk_category', 'GEOID10','iso_week', 'iso_year']).count()

In [54]:
group.head()

rat_violation
establishment_type risk_category GEOID10         iso_week iso_year               
Bakery             1             110010030001000 35       2013                  1
                                 110010093021012 21       2011                  1
                   2             110010001004005 2        2015                  1
                                 110010002023013 7        2017                  1
                                                 21       2014                  2

In [58]:
final = group.reset_index()

In [71]:
#df.rename(index=str, columns={"A": "a", "B": "c"})
final_df = final.rename(index=str, columns={'rat_violation': 'value', 
                                            'establishment_type': 'feature_type',
                                           'risk_category': 'feature_subtype',
                                           'iso_year': 'year',
                                           'iso_week': 'week',
                                           'GEOID10': 'census_block_2010'})

In [73]:
df_final = final_df['feature_id'] = 'restaurant_violations_rodent_or_trash'

In [74]:
final_df

,feature_type,feature_subtype,census_block_2010,week,year,value,feature_id
0,Bakery,1,110010030001000,35,2013,1,restaurant_violations_rodent_or_trash
1,Bakery,1,110010093021012,21,2011,1,restaurant_violations_rodent_or_trash
2,Bakery,2,110010001004005,2,2015,1,restaurant_violations_rodent_or_trash
3,Bakery,2,110010002023013,7,2017,1,restaurant_violations_rodent_or_trash
4,Bakery,2,110010002023013,21,2014,2,restaurant_violations_rodent_or_trash
5,Bakery,2,110010002023013,25,2015,1,restaurant_violations_rodent_or_trash
6,Bakery,2,110010002023013,27,2017,1,restaurant_violations_rodent_or_trash
7,Bakery,2,110010018042007,31,2012,1,restaurant_violations_rodent_or_trash
8,Bakery,2,110010018042007,34,2010,1,restaurant_violations_rodent_or_trash
9,Bakery,2,110010018042007,40,2014,1,restaurant_violations_rodent_or_trash


In [75]:
final_df.to_csv('restaurant_violations_rodent_or_trash.csv')